In [7]:
import ee
import subprocess
import time

# Initialize EE
ee.Authenticate()
ee.Initialize()

# === CONFIGURATION ===
asset_folder = 'projects/proj76/assets/SAR_2021_22/'  # Your EE asset folder
drive_folder = 'Cropland_Mask_2021_22'                  # Drive folder to save output
start_date = '2021-04-01'
end_date = '2022-03-31'

# === 1. Use CLI to list assets ===
def list_assets_in_ee_folder(folder_path):
    result = subprocess.run(['earthengine', 'ls', folder_path], capture_output=True, text=True)
    return [line.strip() for line in result.stdout.split('\n') if line.strip()]

# === 2. Create crop/tree mask ===
def get_valid_land_mask(start_date, end_date):
    dw = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1") \
        .filterDate(start_date, end_date) \
        .select("label")

    mode_label = dw.reduce(ee.Reducer.mode())

    # Define invalid land classes
    invalid_classes = ee.List([0, 6, 7, 8])  # Water, Built Area, Bare Ground, Snow/Ice

    # Create a binary mask: 1 for valid, 0 for invalid
    valid_mask = mode_label.remap(invalid_classes, ee.List.repeat(0, 4), 1).eq(1)

    return valid_mask.rename("valid_mask")


# === 3. Filter FeatureCollection using the mask ===
def filter_points(asset_id, mask):
    fc = ee.FeatureCollection(asset_id)
    
    def keep_if_valid(f):
        sample = mask.sample(region=f.geometry(), scale=10, numPixels=1, geometries=False).first()
        return ee.Algorithms.If(sample, ee.Feature(f), None)

    filtered = fc.map(keep_if_valid).filter(ee.Filter.notNull(['system:index']))
    return filtered

# === 4. Export each filtered FC to Google Drive ===
def export_filtered_fc(asset_id, mask):
    try:
        print(f"🚀 Processing {asset_id}")
        filtered_fc = filter_points(asset_id, mask)

        asset_name = asset_id.split("/")[-1]
        task = ee.batch.Export.table.toDrive(
            collection=filtered_fc,
            description=f"{asset_name}_Filtered",
            folder=drive_folder,
            fileNamePrefix=f"{asset_name}_Masked",
            fileFormat="CSV"
        )
        task.start()
        print(f"✅ Export started: {asset_name}")
        time.sleep(2)
    except Exception as e:
        print(f"❌ Failed for {asset_id}: {e}")

# === MAIN EXECUTION ===
asset_ids = list_assets_in_ee_folder(asset_folder)
print(f"📂 Found {len(asset_ids)} assets in: {asset_folder}")

mask = get_valid_land_mask(start_date, end_date)

for asset_id in asset_ids:
    export_filtered_fc(asset_id, mask)


📂 Found 125 assets in: projects/proj76/assets/SAR_2021_22/
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Bidar_10_SAR_Data_part1
✅ Export started: 2021_Bidar_10_SAR_Data_part1
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Bidar_10_SAR_Data_part2
✅ Export started: 2021_Bidar_10_SAR_Data_part2
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Bidar_1_SAR_Data_part1
✅ Export started: 2021_Bidar_1_SAR_Data_part1
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Bidar_1_SAR_Data_part2
✅ Export started: 2021_Bidar_1_SAR_Data_part2
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Bidar_2_SAR_Data_part1
✅ Export started: 2021_Bidar_2_SAR_Data_part1
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Bidar_2_SAR_Data_part2
✅ Export started: 2021_Bidar_2_SAR_Data_part2
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Bidar_3_SAR_Data_part1
✅ Export started: 2021_Bidar_3_SAR_Data_part1
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Bidar_3_SAR_Data_part2
✅ Expor

✅ Export started: 2021_Dharwad_7_SAR_Data_part1
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Dharwad_7_SAR_Data_part2
✅ Export started: 2021_Dharwad_7_SAR_Data_part2
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Dharwad_8_SAR_Data_part1
✅ Export started: 2021_Dharwad_8_SAR_Data_part1
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Dharwad_8_SAR_Data_part2
✅ Export started: 2021_Dharwad_8_SAR_Data_part2
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Dharwad_9_SAR_Data_part1
✅ Export started: 2021_Dharwad_9_SAR_Data_part1
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Dharwad_9_SAR_Data_part2
✅ Export started: 2021_Dharwad_9_SAR_Data_part2
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Raichur_10_SAR_Data_part1
✅ Export started: 2021_Raichur_10_SAR_Data_part1
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Raichur_10_SAR_Data_part2
✅ Export started: 2021_Raichur_10_SAR_Data_part2
🚀 Processing projects/proj76/assets/SAR_2021_22/2021_Raichur_11_SA